<a href="https://colab.research.google.com/github/vallirajasekar/Cricket_Chatbot/blob/main/Cricket_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional


In [2]:
urls = [
    "https://en.wikipedia.org/wiki/Indian_Premier_League","https://www.iplt20.com/","https://www.cricbuzz.com/","https://www.google.com/search?q=cricinfo&oq=&aqs=chrome.2.69i57j69i64j69i59j0i67i650j0i20i131i263i433i512j0i20i263i433i512j0i131i433i512j0i512.5015j0j15&sourceid=chrome&ie=UTF-8","https://www.espncricinfo.com/cricket-news"
]


In [3]:
def fetch_data(urls):
    raw_docs = []
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text(separator=" ")
        raw_docs.append(text.lower())
    return raw_docs


In [4]:
raw_docs = fetch_data(urls)


In [5]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('wordnet')

# Create a WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

sent_tokens = []
word_tokens = []

for doc in raw_docs:
    sentences = sent_tokenize(doc)
    sent_tokens.extend(sentences)
    words = [lemmatizer.lemmatize(word) for word in word_tokenize(doc)]
    word_tokens.extend(words)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Prepare the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sent_tokens)
total_words = len(tokenizer.word_index) + 1


In [8]:
input_sequences = []
for sentence in sent_tokens:
    seq = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(seq)):
        n_gram_sequence = seq[:i+1]
        input_sequences.append(n_gram_sequence)


In [9]:
max_sequence_len = max([len(seq) for seq in input_sequences])
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')


In [10]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# ...

x_train = padded_sequences[:, :-1]
y_train = padded_sequences[:, -1]
y_train = to_categorical(y_train, num_classes=total_words)

# ...


In [11]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, verbose=1)


Epoch 1/100
514/514 [==============================] - 128s 238ms/step - loss: 6.8728 - accuracy: 0.0565
Epoch 2/100
514/514 [==============================] - 84s 163ms/step - loss: 6.1219 - accuracy: 0.0979
Epoch 3/100
514/514 [==============================] - 70s 135ms/step - loss: 5.6480 - accuracy: 0.1233
Epoch 4/100
514/514 [==============================] - 62s 121ms/step - loss: 5.1757 - accuracy: 0.1563
Epoch 5/100
514/514 [==============================] - 61s 118ms/step - loss: 4.6644 - accuracy: 0.1949
Epoch 6/100
514/514 [==============================] - 57s 110ms/step - loss: 4.2031 - accuracy: 0.2304
Epoch 7/100
514/514 [==============================] - 55s 107ms/step - loss: 3.7822 - accuracy: 0.2695
Epoch 8/100
514/514 [==============================] - 53s 104ms/step - loss: 3.3934 - accuracy: 0.3143
Epoch 9/100
514/514 [==============================] - 55s 108ms/step - loss: 3.0365 - accuracy: 0.3663
Epoch 10/100
514/514 [==============================] - 54s 105

In [ ]:
import random

# Function to generate a response from the model
def generate_response(model, tokenizer, max_sequence_len, input_text):
    input_text = input_text.lower()
    tokenized_text = tokenizer.texts_to_sequences([input_text])[0]
    tokenized_text = pad_sequences([tokenized_text], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(tokenized_text, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    return output_word


In [ ]:
# Function to generate a response from the model
def generate_response(model, tokenizer, max_sequence_len, input_text):
    input_text = input_text.lower()
    tokenized_text = tokenizer.texts_to_sequences([input_text])[0]
    tokenized_text = pad_sequences([tokenized_text], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(tokenized_text)[0]
    predicted_index = tf.argmax(predicted_probs).numpy()
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    return output_word


In [ ]:
def chatbot_demo(model, tokenizer, max_sequence_len):
    print("Chatbot Demo")
    print("Enter 'exit' to end the conversation.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = generate_response(model, tokenizer, max_sequence_len, user_input)
        print("Chatbot:", response)


In [1]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model_name = 'model'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


In [2]:
def generate_response(model, tokenizer, input_text):
    input_text = input_text.lower()
    encoded_input = tokenizer.encode_plus(input_text, add_special_tokens=True, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
    input_ids = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
    
    predicted_index = predicted_labels.item()
    output_word = tokenizer.convert_ids_to_tokens(predicted_index)
    
    return output_word


In [3]:
def chatbot_demo(model, tokenizer):
    print("Chatbot Demo")
    print("Enter 'exit' to end the conversation.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        elif user_input.lower() in ['hi', 'hello', 'hey']:
            print("Chatbot: Hi there!")
        elif '?' in user_input:
            print("Chatbot: I'm sorry, I don't have the answer to that question.")
        else:
            response = generate_response(model, tokenizer, user_input)
            print("Chatbot:", response)


In [5]:
chatbot_demo(model, tokenizer)


NameError: ignored

In [17]:
chatbot_demo(model, tokenizer, max_sequence_len)


Chatbot Demo
Enter 'exit' to end the conversation.
User: KKR
1/1 [==============================] - 0s 52ms/step
Chatbot: final
User: Hi
1/1 [==============================] - 0s 50ms/step
Chatbot: 134
User: CSK
1/1 [==============================] - 0s 51ms/step
Chatbot: v
User: Chennai
1/1 [==============================] - 0s 50ms/step
Chatbot: super
User: Kings
1/1 [==============================] - 0s 52ms/step
Chatbot: out
User: cricinfo
1/1 [==============================] - 0s 52ms/step
Chatbot: google
User: dhoni
1/1 [==============================] - 0s 55ms/step
Chatbot: back
User: rohit
1/1 [==============================] - 0s 52ms/step
Chatbot: needed
User: virat
1/1 [==============================] - 0s 51ms/step
Chatbot: today
User: ipl 2022
1/1 [==============================] - 0s 51ms/step
Chatbot: records
User: records
1/1 [==============================] - 0s 66ms/step
Chatbot: the


KeyboardInterrupt: ignored